In [1]:
import torch
path = '/home/jgryu/workspace/weight_compression/hf_model_comp/comp_qtip/ckpt/meta-llama--Meta-Llama-3-8B/ql_ldlq128_rnorm_codes_test/lmbda50/0_up.pt'
path = '/workspace/Weight_compression/hf_model_comp/qtip/hf/qwen3moe/ft1/2bit.pth'
s = torch.load(path, weights_only=False, map_location='cpu')

In [2]:
s

OrderedDict([('model.embed_tokens.weight',
              tensor([[-0.0087, -0.0109,  0.0011,  ...,  0.0043, -0.0046, -0.0251],
                      [-0.0085, -0.0146, -0.0124,  ...,  0.0099,  0.0010,  0.0007],
                      [-0.0112, -0.0112,  0.0098,  ..., -0.0092, -0.0063, -0.0091],
                      ...,
                      [-0.0114, -0.0058,  0.0010,  ...,  0.0007,  0.0071, -0.0065],
                      [-0.0024,  0.0112,  0.0019,  ...,  0.0058, -0.0049, -0.0023],
                      [-0.0106, -0.0075, -0.0055,  ..., -0.0101,  0.0015, -0.0101]],
                     dtype=torch.bfloat16)),
             ('model.layers.0.self_attn.q_proj.tlut',
              tensor([[-0.8633, -1.0391],
                      [-0.1846, -0.4941],
                      [ 0.8477,  0.6406],
                      ...,
                      [ 1.2188, -0.8438],
                      [-1.6406,  0.3164],
                      [ 0.7656, -1.8438]], dtype=torch.bfloat16)),
             ('model.l

### NWC state_dict 만들기

In [85]:
import torch
from torch.nn.utils.rnn import pad_sequence

path = '/home/jgryu/workspace/weight_compression/hf_model_comp/comp_qtip/ckpt/meta-llama--Meta-Llama-3-8B/ql_ldlq128_rnorm_codes_test/lmbda50'

codes_dict = {}
metadata_dict = {}

for i in range(32):
    for m in ['q', 'k', 'v', 'o', 'up', 'down', 'gate']:
        pt = f'{path}/{i}_{m}.pt'
        s = torch.load(pt, weights_only=False, map_location='cpu')

        codes = s['codes']
        tensor_code_list = []
        for c in codes:
            # print(len(c['strings'][0][0]))
            byte_stream = c['strings'][0][0]
            tensor_view = torch.frombuffer(byte_stream, dtype=torch.uint8)
            tensor_code_list.append(tensor_view)
            # print(tensor_view.shape)
        
        padded_code_tensor = pad_sequence(tensor_code_list, batch_first=True, padding_value=0)
        
        key = f'{i}_{m}'
        
        codes_dict[key] = padded_code_tensor
        
        metadata_dict[key] = {
            'row_std': s['metadata']['row_std'],
            'qlevel': s['metadata']['qlevel'],
            'shape': (128, int(c['shape'][0] / 128 * 16))
        }

# 4. 두 딕셔너리를 별도의 파일로 저장
torch.save(codes_dict, './8b_compressed_codes.pth')
torch.save(metadata_dict, './8b_compressed_metadata.pth')

In [90]:
from transformers import LlamaForCausalLM

path = '/home/jgryu/workspace/weight_compression/Wparam_dataset/hf_model/meta-llama--Meta-Llama-3-8B'
model = LlamaForCausalLM.from_pretrained(
    path,
    torch_dtype=torch.float16,
    device_map='cpu')

torch.save(model.state_dict(), './8b_sd_original.pth')

Loading checkpoint shards: 100%|██████████| 7/7 [00:04<00:00,  1.51it/s]


### 시간측정 함수 정의

In [93]:
import time
import torch
import os
import argparse

def measure_load_time(model_path, device):
    """
    .pth 파일을 로드하고 GPU로 전송하는 시간을 측정합니다.
    이 함수는 캐시 상태(hot/cold)를 알지 못합니다.
    """


    # --- 1. Disk -> CPU RAM 시간 측정 (CPU 작업) ---
    # (Cold Load 시: Disk I/O + Pickle 역직렬화)
    # (Hot Load 시: OS Cache I/O + Pickle 역직렬화)
    
    torch.cuda.synchronize(device) # 이전 GPU 작업 완료 보장
    start_cpu_time = time.perf_counter()
    
    # torch.load는 I/O와 역직렬화(CPU 작업)를 수행합니다.
    state_dict_cpu = torch.load(model_path, map_location='cpu', weights_only=False)
        
    end_cpu_time = time.perf_counter()
    torch.cuda.synchronize(device)
    
    load_time_ms = (end_cpu_time - start_cpu_time) * 1000
    print(f"  [1] Disk/Cache -> CPU RAM 시간: {load_time_ms:.2f} ms")


    if not os.path.exists(model_path):
        print(f"오류: {model_path} 파일을 찾을 수 없습니다.")
        return

    if device.type == 'cpu':
        print("오류: CUDA 디바이스가 필요합니다.")
        return

    # --- 2. CPU RAM -> GPU VRAM 시간 측정 (GPU 작업) ---
    # GPU는 비동기로 작동하므로 정확한 측정을 위해 cuda.Event 사용
    start_gpu_event = torch.cuda.Event(enable_timing=True)
    end_gpu_event = torch.cuda.Event(enable_timing=True)
    
    start_gpu_event.record()
    
    # state_dict의 모든 텐서를 GPU로 이동시킵니다.
    state_dict_gpu = {k: v.to(device) for k, v in state_dict_cpu.items()}
    
    end_gpu_event.record()
    
    # GPU 작업이 끝날 때까지 CPU가 기다리도록 강제
    torch.cuda.synchronize(device)
    
    transfer_time_ms = start_gpu_event.elapsed_time(end_gpu_event)
    print(f"  [2] CPU RAM -> GPU VRAM 시간: {transfer_time_ms:.2f} ms")
    print(f"  -------------------------------------")
    print(f"  총합: {load_time_ms + transfer_time_ms:.2f} ms")
    
    # # 측정 후 메모리 정리
    del state_dict_cpu, state_dict_gpu
    torch.cuda.empty_cache()

In [ ]:
paths = [
    '8b_sd_original.pth',
    '8b_compressed_codes.pth'
]
for p in paths:
    print(p)
    measure_load_time(p, torch.device('cuda:1'))

8b_sd_original.pth
  [1] Disk/Cache -> CPU RAM 시간: 5166.80 ms
  [2] CPU RAM -> GPU VRAM 시간: 1769.98 ms
  -------------------------------------
  총합: 6936.78 ms
8b_compressed_codes.pth
  [1] Disk/Cache -> CPU RAM 시간: 315.34 ms
  [2] CPU RAM -> GPU VRAM 시간: 286.43 ms
  -------------------------------------
  총합: 601.78 ms


: 